## Imports

In [99]:
import pandas as pd

In [100]:
df_awards_players = pd.read_csv('data/awards_players_processed.csv')
df_coaches = pd.read_csv('data/coaches_processed.csv')
df_players_teams = pd.read_csv('data/players_teams_processed.csv')
df_players = pd.read_csv('data/players_processed.csv')
df_series_post = pd.read_csv('data/series_post_processed.csv')
df_teams = pd.read_csv('data/teams_processed.csv')
df_teams_post = pd.read_csv('data/teams_post_processed.csv')

## Merge datasets

Agregate the data from series_post by the number of wins of each player and the number of losses, then join it to the Teams dataset 

In [101]:

# Join the team statistics with the second DataFrame
df_teams = pd.merge(df_teams, df_series_post, on=["year", "tmID"], how="left").fillna(0)

df_teams.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          142 non-null    int64  
 1   tmID          142 non-null    object 
 2   playoff       142 non-null    int64  
 3   confID_EA     142 non-null    float64
 4   confID_WE     142 non-null    float64
 5   total_wins    142 non-null    float64
 6   total_losses  142 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 7.9+ KB


Agregate data from players team, players info and awards and join it to the Teams dataset

In [102]:
players_teams_info_df = pd.merge(df_players_teams, df_players, left_on='playerID', right_on='bioID', how='left')
players_teams_info_df = pd.merge(players_teams_info_df, df_awards_players, on=['playerID', 'year'], how='left').fillna(0)
players_teams_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   playerID            1290 non-null   object 
 1   year                1290 non-null   int64  
 2   tmID                1290 non-null   object 
 3   GP                  1290 non-null   float64
 4   GS                  1290 non-null   float64
 5   minutes             1290 non-null   float64
 6   points              1290 non-null   float64
 7   oRebounds           1290 non-null   float64
 8   dRebounds           1290 non-null   float64
 9   rebounds            1290 non-null   float64
 10  assists             1290 non-null   float64
 11  steals              1290 non-null   float64
 12  blocks              1290 non-null   float64
 13  turnovers           1290 non-null   float64
 14  PF                  1290 non-null   float64
 15  fgAttempted         1290 non-null   float64
 16  fgMade

For each team and year agregate the statistics of the players in the team and join it to the Teams dataset.

In [103]:
agg_players_df = players_teams_info_df.groupby(["year", "tmID"]).agg({
    "GP": "mean",
    "GS": "mean",
    "minutes": "mean",
    "points": "mean",
    "oRebounds": "mean",
    "dRebounds": "mean",
    "rebounds": "mean",
    "assists": "mean",
    "steals": "mean",
    "blocks": "mean",
    "turnovers": "mean",
    "PF": "mean",
    "fgAttempted": "mean",
    "fgMade": "mean",
    "ftAttempted": "mean",
    "ftMade": "mean",
    "threeAttempted": "mean",
    "threeMade": "mean",
    "dq": "mean",
    "PostGP": "mean",
    "PostGS": "mean",
    "PostMinutes": "mean",
    "PostPoints": "mean",
    "PostoRebounds": "mean",
    "PostdRebounds": "mean",
    "PostRebounds": "mean",
    "PostAssists": "mean",
    "PostSteals": "mean",
    "PostBlocks": "mean",
    "PostTurnovers": "mean",
    "PostPF": "mean",
    "PostfgAttempted": "mean",
    "PostfgMade": "mean",
    "PostftAttempted": "mean",
    "PostftMade": "mean",
    "PostthreeAttempted": "mean",
    "PostthreeMade": "mean",
    "PostDQ": "mean",
    "height": "mean",
    "weight": "mean",
    "award_count": "mean",

    # Add more columns you want to aggregate here
}).reset_index()

# agg_players_df['GP'] = agg_players_df['GP'] + agg_players_df['PostGP']
# agg_players_df['GS'] = agg_players_df['GS'] + agg_players_df['PostGS']
# agg_players_df['minutes'] = agg_players_df['minutes'] + agg_players_df['PostMinutes']
# agg_players_df['points'] = agg_players_df['points'] + agg_players_df['PostPoints']
# agg_players_df['oRebounds'] = agg_players_df['oRebounds'] + agg_players_df['PostoRebounds']
# agg_players_df['dRebounds'] = agg_players_df['dRebounds'] + agg_players_df['PostdRebounds']
# agg_players_df['rebounds'] = agg_players_df['rebounds'] + agg_players_df['PostRebounds']
# agg_players_df['assists'] = agg_players_df['assists'] + agg_players_df['PostAssists']
# agg_players_df['steals'] = agg_players_df['steals'] + agg_players_df['PostSteals']
# agg_players_df['blocks'] = agg_players_df['blocks'] + agg_players_df['PostBlocks']
# agg_players_df['turnovers'] = agg_players_df['turnovers'] + agg_players_df['PostTurnovers']
# agg_players_df['PF'] = agg_players_df['PF'] + agg_players_df['PostPF']
# agg_players_df['fgAttempted'] = agg_players_df['fgAttempted'] + agg_players_df['PostfgAttempted']
# agg_players_df['fgMade'] = agg_players_df['fgMade'] + agg_players_df['PostfgMade']
# agg_players_df['ftAttempted'] = agg_players_df['ftAttempted'] + agg_players_df['PostftAttempted']
# agg_players_df['ftMade'] = agg_players_df['ftMade'] + agg_players_df['PostftMade']
# agg_players_df['threeAttempted'] = agg_players_df['threeAttempted'] + agg_players_df['PostthreeAttempted']
# agg_players_df['threeMade'] = agg_players_df['threeMade'] + agg_players_df['PostthreeMade']
# agg_players_df['dq'] = agg_players_df['dq'] + agg_players_df['PostDQ']

#agg_players_df.drop(columns=['PostGP', 'PostGS', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds', 'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks', 'PostTurnovers', 'PostPF', 'PostfgAttempted', 'PostfgMade', 'PostftAttempted', 'PostftMade', 'PostthreeAttempted', 'PostthreeMade', 'PostDQ'], inplace=True)


df_teams = pd.merge(df_teams, agg_players_df, on=['year', 'tmID'], how='left').fillna(0)
#agg_players_df.head()

In [104]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                142 non-null    int64  
 1   tmID                142 non-null    object 
 2   playoff             142 non-null    int64  
 3   confID_EA           142 non-null    float64
 4   confID_WE           142 non-null    float64
 5   total_wins          142 non-null    float64
 6   total_losses        142 non-null    float64
 7   GP                  142 non-null    float64
 8   GS                  142 non-null    float64
 9   minutes             142 non-null    float64
 10  points              142 non-null    float64
 11  oRebounds           142 non-null    float64
 12  dRebounds           142 non-null    float64
 13  rebounds            142 non-null    float64
 14  assists             142 non-null    float64
 15  steals              142 non-null    float64
 16  blocks  

Merge team information with coach information

In [105]:
df_coaches_with_awards = pd.merge(df_coaches, df_awards_players, left_on=['coachID', 'year'], right_on=['playerID', 'year'], how='left').fillna(0)
df_coaches_with_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   coachID      162 non-null    object 
 1   year         162 non-null    int64  
 2   tmID         162 non-null    object 
 3   stint        162 non-null    int64  
 4   won          162 non-null    float64
 5   lost         162 non-null    float64
 6   post_wins    162 non-null    float64
 7   post_losses  162 non-null    float64
 8   playerID     162 non-null    object 
 9   award_count  162 non-null    float64
dtypes: float64(5), int64(2), object(3)
memory usage: 12.8+ KB


In [106]:
agg_coaches_df = df_coaches_with_awards.groupby(["year", "tmID"]).agg({
    "won": "sum",
    "lost": "sum",
    "post_wins": "sum",
    "post_losses": "sum",
    "award_count": "sum",
}).reset_index()

df_teams = pd.merge(df_teams, agg_coaches_df, on=['year', 'tmID'], how='left')
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 53 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                142 non-null    int64  
 1   tmID                142 non-null    object 
 2   playoff             142 non-null    int64  
 3   confID_EA           142 non-null    float64
 4   confID_WE           142 non-null    float64
 5   total_wins          142 non-null    float64
 6   total_losses        142 non-null    float64
 7   GP                  142 non-null    float64
 8   GS                  142 non-null    float64
 9   minutes             142 non-null    float64
 10  points              142 non-null    float64
 11  oRebounds           142 non-null    float64
 12  dRebounds           142 non-null    float64
 13  rebounds            142 non-null    float64
 14  assists             142 non-null    float64
 15  steals              142 non-null    float64
 16  blocks  

Merges the teams with teams post and added the value 0 to the teams that did not have any wins or losses

In [107]:
df_teams = pd.merge(df_teams, df_teams_post, on=['year', 'tmID'], how='left').fillna(0)
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 55 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                142 non-null    int64  
 1   tmID                142 non-null    object 
 2   playoff             142 non-null    int64  
 3   confID_EA           142 non-null    float64
 4   confID_WE           142 non-null    float64
 5   total_wins          142 non-null    float64
 6   total_losses        142 non-null    float64
 7   GP                  142 non-null    float64
 8   GS                  142 non-null    float64
 9   minutes             142 non-null    float64
 10  points              142 non-null    float64
 11  oRebounds           142 non-null    float64
 12  dRebounds           142 non-null    float64
 13  rebounds            142 non-null    float64
 14  assists             142 non-null    float64
 15  steals              142 non-null    float64
 16  blocks  

In [108]:
df_teams.to_csv('data_merged.csv', index=False)